In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from numpy import exp
import json

In [2]:
# Achieves over 90% accuracy on MNIST
data = pd.read_csv("/Users/aryank/Developer/NNs/neural-network-from-scratch/train.csv")
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

In [4]:
# Transpose to make every column an example, instead of row
dev = data[0:1000].T 
Y_dev = dev[0]
X_dev = dev[1:n]
X_dev = X_dev / 255.0 

train = data[1000:m].T
Y_train = train[0]
X_train = train[1:n]
X_train = X_train / 255.0

In [5]:
# First example
X_train[:, 0].shape

(784,)

In [6]:
# Parameters
def init_params():
    W1 = np.random.randn(64, 784) * np.sqrt(2 / 784)
    b1 = np.zeros((64, 1))
    W2 = np.random.randn(10, 64) * np.sqrt(2 / 64)
    b2 = np.zeros((10, 1))
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(0, Z)

def softmax(Z):
    return exp(Z - np.max(Z, axis=0, keepdims=True)) / np.sum(exp(Z - np.max(Z, axis=0, keepdims=True)), axis=0, keepdims=True)

def forward(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

    

In [7]:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def derivative_ReLU(Z):
    return Z > 0

def backprop(Z1, A1, Z2, A2, W2, X, Y):
    m = Y.size
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2, axis=1, keepdims=True)

    dZ1 = W2.T.dot(dZ2) * derivative_ReLU(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, axis=1, keepdims=True)

    return dW1, db1, dW2, db2

In [8]:
def update_wandb(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [9]:
def predictions(A2):
    return np.argmax(A2, 0)

def accuracy(predictions, Y):
    # print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, iterations, alpha):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backprop(Z1, A1, Z2, A2, W2, X, Y)
        W1, b1, W2, b2 = update_wandb(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)

        if i % 10 == 0:
            print(f"i: {i}, Acc: {accuracy(predictions(A2), Y)}")

            # save the model checkpoints
            np.savez(f"model_checkpoint_{i}_a2.npz", W1, b1, W2, b2)

    return W1, b1, W2, b2

In [ ]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 1000, 0.05)

# save the model in both npz and json
np.savez('model_a2.npz', W1, b1, W2, b2)
json.dump({'W1': W1.tolist(), 'b1': b1.tolist(), 'W2': W2.tolist(), 'b2': b2.tolist()}, open('model_a2.json', 'w'))

i: 0, Acc: 0.16282926829268293
i: 10, Acc: 0.5472926829268293
i: 20, Acc: 0.7092195121951219
i: 30, Acc: 0.7684146341463415
i: 40, Acc: 0.7971951219512196
i: 50, Acc: 0.8143658536585365
i: 60, Acc: 0.828170731707317
i: 70, Acc: 0.8371219512195122
i: 80, Acc: 0.8455609756097561
i: 90, Acc: 0.8521219512195122
i: 100, Acc: 0.8571219512195122
i: 110, Acc: 0.8619024390243902
i: 120, Acc: 0.8661951219512195
i: 130, Acc: 0.8693170731707317
i: 140, Acc: 0.8720975609756098
i: 150, Acc: 0.8745121951219512
i: 160, Acc: 0.8768048780487805
i: 170, Acc: 0.8786829268292683
i: 180, Acc: 0.8803658536585366
i: 190, Acc: 0.8818292682926829
i: 200, Acc: 0.8833170731707317
i: 210, Acc: 0.8845365853658537
i: 220, Acc: 0.8862682926829268
i: 230, Acc: 0.887
i: 240, Acc: 0.8880731707317073
i: 250, Acc: 0.8892926829268293
i: 260, Acc: 0.8900487804878049
i: 270, Acc: 0.891219512195122
i: 280, Acc: 0.8921219512195122
i: 290, Acc: 0.8932439024390244
i: 300, Acc: 0.8942682926829268
i: 310, Acc: 0.8953414634146342
i